# Docker Container

1. Устанавливаем `Docker`
2. Скачиваем контейнер 
> docker pull bigtop/sandbox:1.2.1-ubuntu-16.04-hdfs_yarn_hive_pig
3. Запускаем:
    > docker run --hostname=quickstart --name=hadoop-sandbox --privileged=true \    
    > -p 50070:50070 -p 14000:14000 -p 8088:8088 -p 19888:19888 \   
    > -p 9083:9083 -p 10000:10000 -p 10002:10002 \   
    > bigtop/sandbox:1.2.1-ubuntu-16.04-hdfs_yarn_hive_pig
4. Запускаем терминал
    > docker exec -it hadoop-sandbox bash
6. В консоле
    > apt update -y & apt install -y python3 python3-pip hadoop-httpfs  
    > service hadoop-httpfs start 
    >
    > python3 -m pip install mrjob



### Сервисы:
- http://localhost:50070 - HDFS WEB UI
- http://localhost:14000 - HttpFS REST API
- http://localhost:8088 - Resource Manager
- http://localhost:19888 - Hadoop History Server

# HDFS

Hadoop Distributed File System - распределенная файловая система. Запускалась на кластере ~ 4000 машин. 
- устойчива к падениям
- масштабируема 
- безопасность (права доступа и т.д.)
- написана на Java
- концепция основана на Google GFS
- на узлах кластера используются нативные файловые системы


Узлы бывают двух видов:
- `NameNode` - хранение метаинформации
- `DataNode` - хранение данных

Бывает `Secondary NameNode`.

В целом процесс получения/записи данных выглядит так:
1. Клиент обращается к `NameNode`
2. Клиент перенаправляется к соответствующему `DataNone` для  чтения/записи данных. 


![HDFS Architecture](https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-hdfs/images/hdfsarchitecture.png)

Данные реплицируются, `NameNode` постоянно общается с `DataNode` и отслеживает их состояние, перераспределяет данные. `DataNode` могут обмениваться данными напрямую.

![](https://hadoop.apache.org/docs/r1.2.1/images/hdfsdatanodes.gif)

Логически HDFS представляет из себя привычную иерархическую файловую систему. 

- файлы делятся на блоки (по умолячанию 128Мб)
- каждый блок файла копируется по узлам сети (по умолчанию на 3 уззла)
- `NameNode` отслеживает каждый состояние узлов
- если узел стал недоступен, то блок автоматически до-реплицируется

# Python API

HDFS представляет REST HTTP API, для которого есть клиенты, например, из `Python`

In [2]:
import hdfs

http_fs = hdfs.InsecureClient('http://localhost:14000', user='hdfs')

with http_fs.write('lines.txt', encoding='utf-8') as w:
    for i in range(200):
        w.write(str(i))
        w.write('\n')

In [3]:
http_fs.delete('lines.txt')

True

In [4]:
http_fs.upload('weather.csv', 'data/weather.csv')

'/user/hdfs/weather.csv'

In [5]:
http_fs.delete('weather.csv')

True